In [3]:
from dolfinx import *
from dolfinx import default_scalar_type
from dolfinx.fem import *
from mpi4py import *
from dolfinx.fem.petsc import LinearProblem
from dolfinx.plot import vtk_mesh
import numpy as np
import matplotlib
import matplotlib.pyplot as pl
import math
import pyvista
import ufl
import basix

In [26]:
def v_exact_np(x):
    u = 1
    r = np.sqrt((x[0])**2+(x[1])**2)
    theta = np.atan2(x[1],x[0])

    d_psi_d_r = (u/((1/4)*(np.pi**2)-1))*((-1/4)*(np.pi**2)*(np.sin(theta))+((1/2)*np.pi*theta*np.sin(theta))+(theta*np.cos(theta)))
    d_psi_d_theta = ((np.pi*theta*np.cos(theta))/2) - (theta*np.sin(theta)) + ((np.pi/2)*np.sin(theta)) + (((- np.pi**2 + 4)/4)) * np.cos(theta)

    return np.array([((np.cos(theta) * d_psi_d_theta/r) + (np.sin(theta) * d_psi_d_r)), ((np.sin(theta) * d_psi_d_theta/r) - (np.cos(theta) * d_psi_d_r))])

In [32]:
#specify function inputs to test
ne = 4
p = 1

# define the domain (unit square) of length and width ne, running in parallel
domain = mesh.create_unit_square(MPI.COMM_WORLD, ne, ne)

v_e = basix.ufl.element("Lagrange", domain.basix_cell(), p+1, shape=(domain.geometry.dim,))
p_e = basix.ufl.element("Lagrange", domain.basix_cell(), p)

vPe = basix.ufl.mixed_element([v_e, p_e])

V = functionspace(domain, vPe)

v_v = V.sub(0)
v_p = V.sub(1)

v_t, p_t = ufl.TestFunctions(V)